## 📊 LangSmith功能

使用 LangChain 构建的许多应用程序将包含多个步骤和多个 LLM 调用的调用。

随着这些应用程序变得越来越复杂，能够检查链或代理内部到底发生了什么变得至关重要
https://smith.langchain.com/

1. **📈 性能监控**: 追踪每次 LLM 调用的延迟、token 使用量、成本等
2. **🔍 调试工具**: 查看完整的输入输出链路，便于调试复杂的 LangChain 应用
3. **📝 日志记录**: 自动记录所有 LLM 交互，包括 prompt、响应、元数据等
4. **🏷️ 标签管理**: 通过项目、会话、标签等维度组织和筛选追踪数据
5. **📊 数据分析**: 提供丰富的图表和统计信息，帮助优化应用性能

| 环境变量 | 说明 | 示例值 |
|---------|------|--------|
| `LANGSMITH_TRACING` | 启用/禁用追踪 | `"true"` 或 `"false"` |
| `LANGSMITH_API_KEY` | LangSmith API 密钥 | `"lsv2_pt_..."` |
| `LANGSMITH_PROJECT` | 项目名称 | `"LangChain-Chatbot-20241018"` |
| `LANGSMITH_SESSION` | 会话标识 | `"chatbot-session-1"` |
| `LANGSMITH_TAGS` | 标签（逗号分隔） | `"chatbot,demo,glm-4.6"` |
| `LANGSMITH_ENDPOINT` | API 端点 | `"https://api.smith.langchain.com"` |

In [1]:
import os
from datetime import datetime
from langchain_openai import ChatOpenAI

# 🔧 LangSmith 配置 - 用于追踪和监控 LLM 调用
os.environ["LANGSMITH_TRACING"] = "true"  # 启用追踪

# LANGSMITH_API_KEY 已配置
# os.environ["LANGSMITH_API_KEY"] = "@@@"

os.environ["LANGSMITH_PROJECT"] = f"LangChain-Chatbot-{datetime.now().strftime('%Y%m%d%H%M')}"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

print(f"📊 LangSmith 追踪已启用")
print(f"🏷️ 项目名称: {os.environ['LANGSMITH_PROJECT']}")

# 初始化 LLM 模型
# 这里用的智谱的API Key
llm_model = ChatOpenAI(
    model="glm-4.6",
    temperature=0.7,
    openai_api_key=os.getenv("ZAI_API_KEY"), 
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/",
    max_tokens=2000,  # 最大输出 token 数
)

📊 LangSmith 追踪已启用
🏷️ 项目名称: LangChain-Chatbot-202510181726


## 🔍 对比`langchain_core.messages`和`langchain.schema`的HumanMessage

| 比较项   | `langchain_core.messages.HumanMessage`             | `langchain.schema.HumanMessage`             |
| -------- | -------------------------------------------------- | ------------------------------------------- |
| 所属模块 | `langchain-core`（独立核心库）                     | `langchain` 主包的旧版 schema               |
| 设计目的 | 新版消息协议（和 OpenAI API 对齐）                 | 旧版消息格式定义                            |
| 版本支持 | ✅ LangChain ≥ 0.1（推荐）                          | ⚠️ 旧版兼容层，未来可能移除                  |
| 底层类型 | `BaseMessage`（新定义）                            | `BaseMessage`（旧定义）                     |
| 使用场景 | ReAct Agent、LCEL、Runnable、Tool 等新版接口       | 旧式 Chain（如 `ConversationChain`）        |
| 导入示例 | `from langchain_core.messages import HumanMessage` | `from langchain.schema import HumanMessage` |

In [2]:
# from langchain.schema import HumanMessage
from langchain_core.messages import HumanMessage

llm_model.invoke([
    HumanMessage(content="Hi! Knight"),
])

AIMessage(content='\nHail and well met!\n\nYou have summoned a Knight. I am at your service.\n\nMy virtual armor is polished, and I am ready for any quest you have in mind. How may I assist you today?\n\nPerhaps you wish to:\n*   **Learn** about the history of medieval knights and their code of chivalry.\n*   **Hear a tale** of a famous knight from legend or fiction.\n*   **Discuss** the strategy of the knight piece in chess.\n*   Or simply have a **loyal companion** to chat with.\n\nJust state your command, and I shall endeavor to help.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1531, 'prompt_tokens': 10, 'total_tokens': 1541, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'glm-4.6', 'system_fingerprint': None, 'id': '20251018172612e8f853261b3a4c92', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--f837b363-c96f-4182-bdcf-1d6013e20e3d-0'

## 🤖 带有 LangSmith 追踪的聊天机器人实现

下面我们将实现一个完整的聊天机器人，并展示如何使用 LangSmith 追踪其运行情况。

In [3]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import uuid
from typing import Dict

# 📝 创建聊天提示模板
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""
你是一个友好、专业的 AI 助手。请遵循以下原则：
1. 用简洁、清晰的语言回答问题
2. 如果不确定答案，请诚实地说明
3. 保持对话的连贯性和上下文理解
4. 适当使用 emoji 让对话更生动
"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# 存储不同会话的聊天历史
store: Dict[str, BaseChatMessageHistory] = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """获取或创建会话历史"""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 创建带有历史记录的聊天链
chain = prompt | llm_model

# 包装链以支持消息历史
chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("🤖 聊天机器人已初始化完成！")
print("📊 所有对话都将被 LangSmith 自动追踪")

🤖 聊天机器人已初始化完成！
📊 所有对话都将被 LangSmith 自动追踪


### 🎯 测试聊天机器人 - 单轮对话

In [4]:
session_id = str(uuid.uuid4())  # 生成唯一会话 ID
print(f"🆔 会话 ID: {session_id}")

# 第一轮对话
response1 = chatbot.invoke(
    {"input": "你好！我想了解一下 LangChain 的基本概念。"},
    config={"configurable": {"session_id": session_id}}
)

print("👤 用户:", "你好！我想了解一下 LangChain 的基本概念。")
print("🤖 助手:", response1.content[:200] + "..." if len(response1.content) > 200 else response1.content)
print("\n" + "="*50 + "\n")

🆔 会话 ID: 33899279-6380-4abe-a99f-64eca36ab681
👤 用户: 你好！我想了解一下 LangChain 的基本概念。
🤖 助手: 
你好！很高兴为你介绍 LangChain 的基本概念。👋

简单来说，**LangChain 是一个强大的框架，专门用来构建基于大型语言模型（LLM）的应用程序。**

你可以把它想象成一个“瑞士军刀” 🛠️，它提供了各种工具，让你能轻松地将 LLM（如 GPT-4）与其他数据源或功能连接起来，从而实现更复杂、更强大的任务。

下面是 LangChain 的几个核心概念，我会用最简单的方式为你解...




### 🔄 测试聊天机器人 - 多轮对话（带上下文）

In [5]:
response2 = chatbot.invoke(
    {"input": "能详细解释一下 Chain 的概念吗？"},
    config={"configurable": {"session_id": session_id}}
)

print("👤 用户:", "能详细解释一下 Chain 的概念吗？")
print("🤖 助手:", response2.content[:200] + "..." if len(response2.content) > 200 else response2.content)
print("\n" + "="*50 + "\n")

# 第三轮对话
response3 = chatbot.invoke(
    {"input": "给我一个简单的代码示例吧"},
    config={"configurable": {"session_id": session_id}}
)

print("👤 用户:", "给我一个简单的代码示例吧")
print("🤖 助手:", response3.content[:200] + "..." if len(response3.content) > 200 else response3.content)

👤 用户: 能详细解释一下 Chain 的概念吗？
🤖 助手: 
当然可以！我们来深入聊聊 **Chain (链)** 这个核心概念。🧐

如果说 LangChain 是一个“瑞士军刀”，那么 **Chain 就是使用这些工具的“标准操作流程”**。

---

### 1. Chain 的核心思想：串联与组合

想象一下工厂里的流水线 🏭：
*   原材料（比如用户的输入）进入流水线。
*   经过第一个工位（比如 **提示模板**），被塑造成特定的格式。
...


👤 用户: 给我一个简单的代码示例吧
🤖 助手: 
没问题！我们来写一个最简单、最经典的“Hello World”级别的例子：**一个简单的翻译链**。👍

这个例子会把你输入的任何中文句子，翻译成英文。

---

### 第 1 步：环境准备

在运行代码前，请确保你已经安装了必要的库并设置了 API 密钥。

```bash
# 安装 LangChain 和 OpenAI 的包
pip install langchain langchain...


### 📊 查看当前会话的聊天历史

In [6]:
history = get_session_history(session_id)
print(f"📝 当前会话共有 {len(history.messages)} 条消息：\n")

for i, message in enumerate(history.messages, 1):
    role = "👤 用户" if isinstance(message, HumanMessage) else "🤖 助手"
    content = message.content[:100] + "..." if len(message.content) > 100 else message.content
    print(f"{i}. {role}: {content}\n")

📝 当前会话共有 6 条消息：

1. 👤 用户: 你好！我想了解一下 LangChain 的基本概念。

2. 🤖 助手: 
你好！很高兴为你介绍 LangChain 的基本概念。👋

简单来说，**LangChain 是一个强大的框架，专门用来构建基于大型语言模型（LLM）的应用程序。**

你可以把它想象成一个“瑞士军...

3. 👤 用户: 能详细解释一下 Chain 的概念吗？

4. 🤖 助手: 
当然可以！我们来深入聊聊 **Chain (链)** 这个核心概念。🧐

如果说 LangChain 是一个“瑞士军刀”，那么 **Chain 就是使用这些工具的“标准操作流程”**。

---

...

5. 👤 用户: 给我一个简单的代码示例吧

6. 🤖 助手: 
没问题！我们来写一个最简单、最经典的“Hello World”级别的例子：**一个简单的翻译链**。👍

这个例子会把你输入的任何中文句子，翻译成英文。

---

### 第 1 步：环境准备

...



### 🎯 高级功能：带标签的追踪

In [7]:
from langsmith import traceable

@traceable(
    name="智能客服对话",
    tags=["customer_service", "glm-4.6", "production"]
)
def customer_service_chat(user_input: str, session_id: str) -> str:
    """客服聊天功能，带有自定义追踪"""
    response = chatbot.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

# 测试带标签的追踪
customer_session = str(uuid.uuid4())
result = customer_service_chat(
    "我想退货，请问流程是什么？", 
    customer_session
)

print("🏷️ 带标签的追踪已完成")
print(f"📝 响应: {result[:150]}..." if len(result) > 150 else result)
print("\n💡 在 LangSmith 中可以通过 'customer_service' 标签筛选此类对话")

🏷️ 带标签的追踪已完成
📝 响应: 
您好！很抱歉您对购买的商品不满意。退货流程通常会根据您购买商品的平台或商家有所不同，但大体上可以分为以下几个步骤：

📦 **通用退货流程**

1.  **查看退货政策**：首先，请在您的订单详情页或商品页面查看商家的退货政策。重点关注退货时间（如7天无理由）、商品状态要求（如未拆封、不影响二次...

💡 在 LangSmith 中可以通过 'customer_service' 标签筛选此类对话
